In [1]:
import torchvision
import torch
from torch import nn, optim
import torchvision.transforms as transforms
import numpy as np

batch_size=128

transform = transforms.Compose(
    [transforms.ToTensor()])

### torchvision.datasets
torchvision.datasets에는 다양한 dataset들이 존재한다. 그중 대표적인 예시는 다음과 같다.

- MNIST
- CIFAR
- COCO
- ImageNet
- STL10

In [2]:
train_data=torchvision.datasets.MNIST(root="./MNIST_data", train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True)

In [3]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer_1 = nn.Conv2d(1,64,10,1)
        self.act_1 = nn.ReLU()
        
        self.layer_2 = nn.Conv2d(64,64,5,1)
        self.act_2 = nn.ReLU()
        
        self.layer_3 = nn.Conv2d(64,128,5,1)
        self.act_3 = nn.ReLU()
        
        self.layer_4 = nn.Conv2d(128,128,2,1)
        self.act_4 = nn.ReLU()
        
        self.layer_5 = nn.Conv2d(128,256,2,1)
        self.act_5 = nn.ReLU()
        
        self.max_1=nn.MaxPool2d(2,2)
        
        self.layer_6 = nn.Conv2d(256,256,2,1)
        self.act_6 = nn.ReLU()
        
        self.fc_layer_1 = nn.Linear(9*256,1000)
        self.act_7 = nn.ReLU()
        
        self.bnm1=nn.BatchNorm1d(1000)
        
        self.fc_layer_2 = nn.Linear(1000,10)
        self.act_8 = nn.ReLU()

        
    def forward(self, x):
        x = x.view(batch_size,1,28,28)
        out = self.layer_1(x)
        out = self.act_1(out)
        for module in list(self.modules())[2:-5]:
            out = module(out)
        out = out.view(batch_size,-1)
        for module in list(self.modules())[-5:]:
            out = module(out)
        return out

In [4]:
model=Model()
if torch.cuda.is_available():
    model=model.cuda()
criterion=nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [5]:
total_epoch=10

In [7]:
for epoch in range(total_epoch):
    trn_loss=0
    trn_correct=0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        if torch.cuda.is_available():
            inputs=inputs.cuda()
            labels=labels.cuda()
        # grad init
        optimizer.zero_grad()
        # forward propagation
        output= model(inputs)
        # calculate loss
        loss=criterion(output, labels)
        # back propagation 
        loss.backward()
        # weight update
        optimizer.step()
        
        # trn_loss summary
        trn_loss += loss.item()
        
        _, predicted=torch.max(output,1)
        trn_correct+=np.count_nonzero(predicted.cpu().detach()==labels.cpu().detach())
        # del (memory issue)
        del loss
        del output
    print("epoch: {}/{} | trn loss: {:.4f} | trn accuracy: {:.2f}% ".format(
                epoch+1, total_epoch, trn_loss / len(train_loader), 
        trn_correct/(len(train_loader)*batch_size)*100
            ))

epoch: 1/10 | trn loss: 0.1511 | trn accuracy: 95.27% 
epoch: 2/10 | trn loss: 0.0655 | trn accuracy: 98.02% 
epoch: 3/10 | trn loss: 0.0473 | trn accuracy: 98.58% 
epoch: 4/10 | trn loss: 0.0377 | trn accuracy: 98.81% 
epoch: 5/10 | trn loss: 0.0346 | trn accuracy: 98.97% 
epoch: 6/10 | trn loss: 0.0307 | trn accuracy: 99.05% 
epoch: 7/10 | trn loss: 0.0272 | trn accuracy: 99.14% 
epoch: 8/10 | trn loss: 0.0259 | trn accuracy: 99.21% 
epoch: 9/10 | trn loss: 0.0221 | trn accuracy: 99.33% 
epoch: 10/10 | trn loss: 0.0201 | trn accuracy: 99.38% 
